In [1]:
from utils import (
    download_file,
    parsear_archivo,
    random_path,
    calcular_distancias,
    cumple_capacidad,
    dist
)
from tqdm.notebook import tqdm as tqdm
import math

# Descargo y parseo datos

In [2]:
download_file("https://modelosuno.okapii.com/Problemas/problema2022/problema_tres.txt", "problema_tres.txt", override=True)

  0%|          | 0.00/2.66k [00:00<?, ?iB/s]

In [8]:
CAPACIDAD, DIMENSION, DEMANDAS, TIPO_ARISTA, COORDS = parsear_archivo("problema_tres.txt")

In [24]:
COORDS_S = {
    k: (v["x"], v["y"]) for k, v in COORDS.items()
}

# Armo recorridos

In [25]:
class LoopError(Exception):
    pass

In [26]:
import multiprocessing as mp

def generar_camino(initial, dimension, coords_s):
    try:
        seq = [initial]
        seqset = set(seq)

        for _ in range(dimension - 1):
            # while len(seq) < DIMENSION:
            last = seq[-1]
            x0, y0 = coords_s[last]

            best_candidate = None
            best_distance = math.inf

            for k, (x1, y1) in coords_s.items():
                if k in seqset:
                    continue
                d = (x0 - x1) ** 2 + (y0 - y1) ** 2
                if d < best_distance:
                    best_candidate = k
                    best_distance = d

            if best_candidate is None:
                raise LoopError

            seq.append(best_candidate)
            seqset.add(best_candidate)

        return seq
    except LoopError:
        return None

import functools

initial_candidates = list(COORDS_S.keys())
initial_candidates = tuple(initial_candidates)

In [28]:
costos = []
caminos = []

with mp.Pool(24) as p, tqdm(total=len(initial_candidates)) as bar:
    f = functools.partial(
        generar_camino,
        dimension=DIMENSION,
        coords_s=COORDS_S
    )
    rs = p.imap(f, initial_candidates)
    for r in rs:
        bar.update(1)
        if r is not None:
            caminos.append(r)
            costos.append(calcular_distancias(r, COORDS))

  0%|          | 0/100 [00:00<?, ?it/s]

In [33]:
idx_minimo = costos.index(min(costos))
camino_minimo = caminos[idx_minimo]

print(idx_minimo)
print(calcular_distancias(camino_minimo, COORDS))

63
5632.4894935543825


In [57]:
len(caminos)

100

In [36]:
camino_minimo_d = [(x, *COORDS_S[x]) for x in camino_minimo]

In [43]:
def calcular_distancias_l(l):
    total = 0
    _, x0, y0 = l[0]
    for k1, x1, y1 in l[1:]:
        total += math.sqrt((x0 - x1) ** 2 + (y0 - y1) ** 2)
        x0, y0 = x1, y1
    _, xo, yo = l[0]
    _, xf, yf = l[-1]
    total += math.sqrt((xo - xf) ** 2 + (yo - yf) ** 2)
    return total

In [44]:
%%timeit
calcular_distancias_l(camino_minimo_d)

31.3 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [47]:
import time

path = camino_minimo_d

improvement = True
best_path = path
best_cost = calcular_distancias_l(best_path)

start = time.time()

with tqdm() as bar:
    while improvement:
        if time.time() - start > 60 * 60:
            break
        improvement = False
        for i in range(len(best_path) - 1):
            bar.set_description(f"Empiezo desde {i}")
            for k in range(i + 1, len(best_path)):
                bar.update()
                new_path = best_path[0:i]
                new_path.extend(reversed(best_path[i:k + 1]))
                new_path.extend(best_path[k + 1:])

                new_cost = calcular_distancias_l(new_path)

                if new_cost >= best_cost:
                    continue
                else:
                    print(f"Improved from {best_cost} to {new_cost}")
                    best_cost = new_cost
                    best_path = new_path
                    improvement = True
                    break  # improvement found, return to the top of the while loop
            if improvement:
                break

0it [00:00, ?it/s]

Improved from 5632.4894935543825 to 5625.341388018519
Improved from 5625.341388018519 to 5596.654118805673
Improved from 5596.654118805673 to 5596.654118805672
Improved from 5596.654118805672 to 5585.762115588537
Improved from 5585.762115588537 to 5585.7621155885345
Improved from 5585.7621155885345 to 5578.690474819879
Improved from 5578.690474819879 to 5567.535696987123
Improved from 5567.535696987123 to 5525.63423942725
Improved from 5525.63423942725 to 5487.280179547477
Improved from 5487.280179547477 to 5456.864323272974
Improved from 5456.864323272974 to 5411.213493768052


In [49]:
calcular_distancias_l(best_path)

5411.213493768052

In [50]:
seq_str = " ".join(map(str, [x[0] for x in best_path]))

In [51]:
with open("entrega3_greedy_plus_2opt.txt", "w") as f:
    f.write(seq_str)

In [55]:
print([i for i,_,_ in best_path])

[64, 44, 71, 45, 4, 68, 91, 13, 74, 31, 27, 49, 72, 80, 14, 77, 15, 78, 16, 59, 79, 88, 94, 10, 63, 34, 98, 7, 84, 30, 8, 89, 96, 35, 93, 52, 33, 92, 54, 46, 90, 51, 43, 67, 32, 23, 38, 41, 57, 39, 60, 66, 17, 11, 61, 36, 69, 24, 12, 53, 40, 42, 9, 28, 6, 99, 19, 2, 37, 47, 20, 25, 81, 29, 86, 70, 50, 58, 55, 65, 85, 18, 75, 83, 56, 26, 97, 100, 5, 95, 82, 1, 87, 76, 73, 48, 3, 62, 22, 21]
